## RNN to Make a Model Learn Addition and Subtraction

## Model 2 : Encoder-Decoder using RepeatVector

This is a many-to-many sequence-to-sequence (seq2seq) model where the input and output sequences have <br>
the same length. Unlike the traditional encoder-decoder structure (where the encoder compresses an <br>
input into a single latent vector), this model passes the entire sequence through both RNNs.<br>


Given the string "54+7", the model should return a prediction: "61" <br>
Given the string "54-3", the model should return a prediction: "51" <br>

We will use an encoder-decoder architecture



### Load Libraries

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector, Input,LSTM, GRU
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from tensorflow.keras.callbacks import Callback

from termcolor import colored

import sys
sys.stderr = open('err.txt', 'w')

2025-03-09 22:54:43.141503: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 22:54:43.149391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-09 22:54:43.158341: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-09 22:54:43.161024: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 22:54:43.167992: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()
gpu_devices = [device for device in devices if device.device_type == 'GPU']
for gpu in gpu_devices:
    print('Using', gpu.physical_device_desc)

### Generate Data

In [3]:
all_chars    = '0123456789+-'
num_features = len(all_chars)
print('Number of features:', num_features)
trace = True
target_sequence_length = 5

Number of features: 12


In [4]:

char_to_index = dict((c, i) for i, c in enumerate(all_chars))
index_to_char = dict((i, c) for i, c in enumerate(all_chars))



In [5]:
def generate_data():
    first_num = np.random.randint(low=0,high=100)
    second_num = np.random.randint(low=0,high=100)
    add = np.squeeze(np.random.randint(low=0, high=100)) > 50.
    if add:
        example = str(first_num) + '+' + str(second_num)
        label = str(first_num+second_num)
    else:
        example = str(first_num) + '-' + str(second_num)
        label = str(first_num-second_num)
    return example, label

generate_data()

('39-41', '-2')

### Support Functions

### Create the Model

In [6]:
"""

Interesting callback that prints the accuracy on each time step

"""
class SimpleLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            print(f"Epoch {epoch+1}: Loss = {logs['loss']:.4f}, Val Accuracy = {logs.get('val_accuracy', logs.get('val_acc', 'N/A')):.4f}")

simple_logger = SimpleLogger()

In [28]:
# Network Hyperparameters

hidden_units = 64
num_features = 12

model = Sequential([
    Bidirectional(LSTM(64, return_sequences= True, input_shape=(None, num_features))),
    Bidirectional(LSTM(32, return_sequences=True)),                                  
    Dense(num_features, activation='softmax')                                           
])


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_8 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### Vectorize and De-Vectorize Data

In [29]:
def vectorize_example(example, label):
    
    x = np.zeros((target_sequence_length, num_features))
    y = np.zeros((target_sequence_length, num_features))
    
    diff_x = target_sequence_length - len(example)
    diff_y = target_sequence_length - len(label)
    
    for i, c in enumerate(example):
        x[diff_x+i, char_to_index[c]] = 1
    for i in range(diff_x):
        x[i, char_to_index['0']] = 1
    for i, c in enumerate(label):
        y[diff_y+i, char_to_index[c]] = 1
    for i in range(diff_y):
        y[i, char_to_index['0']] = 1
        
    return x, y

e, l = generate_data()
print('Text Example and Label:', e, l)
x, y = vectorize_example(e, l)
print('Vectorized Example and Label Shapes:', x.shape, y.shape)

Text Example and Label: 15-22 -7
Vectorized Example and Label Shapes: (5, 12) (5, 12)


In [30]:
def devectorize_example(example):
    result = [index_to_char[np.argmax(vec)] for i, vec in enumerate(example)]
    return ''.join(result)

def strip_padding(example):
    encountered_non_zero = False
    output = ''
    for c in example:
        if not encountered_non_zero and c == '0':
            continue
        if c == '+' or c == '-':
            encountered_non_zero = False
        else:
            encountered_non_zero = True
        output += c
    return output

devectorize_example(x)

'15-22'

In [31]:
print(devectorize_example(y), ':', strip_padding(devectorize_example(y)))

000-7 : -7


### Create Dataset

In [32]:
def create_dataset(num_examples=2000):

    x_train = np.zeros((num_examples, target_sequence_length, num_features))
    y_train = np.zeros((num_examples, target_sequence_length, num_features))

    for i in range(num_examples):
        e, l = generate_data()
        x, y = vectorize_example(e, l)
        x_train[i] = x
        y_train[i] = y
    
    return x_train, y_train

x_train, y_train = create_dataset(20000)
print(x_train.shape, y_train.shape)

(20000, 5, 12) (20000, 5, 12)


In [33]:
devectorize_example(x_train[0])

'25+98'

In [34]:
devectorize_example(y_train[0])

'00123'

### Training the Model

In [35]:
def see_trained_output (preds):
    full_seq_acc = 0
    for i, pred in enumerate(preds):
        pred_str   = strip_padding(devectorize_example(pred))
        y_test_str = strip_padding(devectorize_example(y_test[i]))
        x_test_str = strip_padding(devectorize_example(x_test[i]))
        col = 'green' if pred_str == y_test_str else 'red'
        full_seq_acc += 1/len(preds) * int(pred_str == y_test_str)
        outstring = 'Input: {}, Out: {}, Pred: {}'.format(x_test_str, y_test_str, pred_str)
        print(colored(outstring, col))
    print('\nFull sequence accuracy: {:.3f} %'.format(100 * full_seq_acc))

In [36]:
""" 
    We train the model and see a random list of 20 examples how it is working 
    you will see the progression of the learning, but only if trace is True
"""
trace = True
for iteration in range(1,200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, epochs=1, validation_split=0.2, verbose=False,
         batch_size=1024, callbacks=[simple_logger])
    x_test, y_test = create_dataset(num_examples=20)
    preds = model.predict(x_test)
    if trace:
        see_trained_output(preds)


--------------------------------------------------
Iteration 1
Epoch 1: Loss = 2.3936, Val Accuracy = 0.5450
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
Input: 26-82, Out: -56, Pred: 
Input: 90+94, Out: 184, Pred: 
Input: 58+54, Out: 112, Pred: 
Input: 34+7, Out: 41, Pred: 
Input: 51-65, Out: -14, Pred: 
Input: 80-86, Out: -6, Pred: 
Input: 38+45, Out: 83, Pred: 
Input: 12+34, Out: 46, Pred: 
Input: 80+28, Out: 108, Pred: 
Input: 82-87, Out: -5, Pred: 
Input: 50-39, Out: 11, Pred: 
Input: 14+80, Out: 94, Pred: 
Input: 68-91, Out: -23, Pred: 
Input: 91-49, Out: 42, Pred: 
Input: 13+76, Out: 89, Pred: 
Input: 46-12, Out: 34, Pred: 
Input: 34+94, Out: 128, Pred: 
Input: 43-55, Out: -12, Pred: 
Input: 8-61, Out: -53, Pred: 
Input: 34-99, Out: -65, Pred: 

Full sequence accuracy: 0.000 %

--------------------------------------------------
Iteration 2
Epoch 1: Loss = 1.8724, Val Accuracy = 0.5450
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Input: 68+25, Out: 93, Pred: 
Input: 47+55, Out: 102, Pred: 
I

Epoch 1: Loss = 1.0889, Val Accuracy = 0.5837
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Input: 55-57, Out: -2, Pred: 13
Input: 93-94, Out: -1, Pred: 12
Input: 54-42, Out: 12, Pred: 3
Input: 48-69, Out: -21, Pred: 12
Input: 99+12, Out: 111, Pred: 117
Input: 81+62, Out: 143, Pred: 17
Input: 61-66, Out: -5, Pred: 12
Input: 99+57, Out: 156, Pred: 117
Input: 27+29, Out: 56, Pred: 117
Input: 43+32, Out: 75, Pred: 13
Input: 72-24, Out: 48, Pred: 3
Input: 26+66, Out: 92, Pred: 117
Input: 44+18, Out: 62, Pred: 13
Input: 21-71, Out: -50, Pred: 2
Input: 1-6, Out: -5, Pred: -4
Input: 54+17, Out: 71, Pred: 13
Input: 72-74, Out: -2, Pred: 13
Input: 41-4, Out: 37, Pred: -4
Input: 58-17, Out: 41, Pred: 12
Input: 4-55, Out: -51, Pred: 3

Full sequence accuracy: 0.000 %

--------------------------------------------------
Iteration 10
Epoch 1: Loss = 1.0659, Val Accuracy = 0.5905
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Input: 60-68, Out: -8, Pred: -12
Input: 42+88, Out: 130, Pred: 116
Input: 66+47, Out: 113, P

Epoch 1: Loss = 0.9852, Val Accuracy = 0.6225
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Input: 45+67, Out: 112, Pred: 114
Input: 91+74, Out: 165, Pred: 114
Input: 82-48, Out: 34, Pred: 11
Input: 4-31, Out: -27, Pred: -31
Input: 60+25, Out: 85, Pred: 
Input: 46+55, Out: 101, Pred: 114
Input: 74-17, Out: 57, Pred: 11
Input: 71+61, Out: 132, Pred: 10
Input: 39-58, Out: -19, Pred: -11
Input: 33+30, Out: 63, Pred: 
Input: 13-7, Out: 6, Pred: 21
Input: 70+23, Out: 93, Pred: 
Input: 73+97, Out: 170, Pred: 114
Input: 94-63, Out: 31, Pred: 12
Input: 16+69, Out: 85, Pred: 179
Input: 52+84, Out: 136, Pred: 129
Input: 40+10, Out: 50, Pred: 
Input: 36-29, Out: 7, Pred: -11
Input: 2-5, Out: -3, Pred: 31
Input: 68-, Out: 68, Pred: 25

Full sequence accuracy: 0.000 %

--------------------------------------------------
Iteration 18
Epoch 1: Loss = 0.9791, Val Accuracy = 0.6236
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Input: 50+71, Out: 121, Pred: 90
Input: 75+93, Out: 168, Pred: 114
Input: 17-16, Out: 1, Pred

Epoch 1: Loss = 0.9284, Val Accuracy = 0.6397
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Input: 99-17, Out: 82, Pred: 
Input: 61+87, Out: 148, Pred: 110
Input: 41-32, Out: 9, Pred: 11
Input: 70-18, Out: 52, Pred: 41
Input: 81+99, Out: 180, Pred: 174
Input: 48+56, Out: 104, Pred: 114
Input: 63+94, Out: 157, Pred: 134
Input: 4-26, Out: -22, Pred: -11
Input: 68+61, Out: 129, Pred: 116
Input: 80-54, Out: 26, Pred: 24
Input: 66+81, Out: 147, Pred: 136
Input: 39+51, Out: 90, Pred: 90
Input: 36-43, Out: -7, Pred: 11
Input: 4+13, Out: 17, Pred: 1
Input: 56-77, Out: -21, Pred: -11
Input: 45-26, Out: 19, Pred: 11
Input: 15-37, Out: -22, Pred: --1
Input: 84-98, Out: -14, Pred: -12
Input: 56-99, Out: -43, Pred: -12
Input: 43-45, Out: -2, Pred: 11

Full sequence accuracy: 5.000 %

--------------------------------------------------
Iteration 26
Epoch 1: Loss = 0.9186, Val Accuracy = 0.6419
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Input: 85-14, Out: 71, Pred: 4
Input: 55-71, Out: -16, Pred: -11
Input: 88-39,

Epoch 1: Loss = 0.8474, Val Accuracy = 0.6816
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Input: 82+27, Out: 109, Pred: 114
Input: 19+81, Out: 100, Pred: 100
Input: 2+9, Out: 11, Pred: 11
Input: 14+50, Out: 64, Pred: 51
Input: 64-87, Out: -23, Pred: -20
Input: 70+25, Out: 95, Pred: 110
Input: 59-5, Out: 54, Pred: 77
Input: 87+39, Out: 126, Pred: 117
Input: 82-31, Out: 51, Pred: 43
Input: 19+83, Out: 102, Pred: 100
Input: 15-4, Out: 11, Pred: 17
Input: 95+69, Out: 164, Pred: 124
Input: 99-32, Out: 67, Pred: 66
Input: 79+45, Out: 124, Pred: 114
Input: 8-, Out: 8, Pred: 7
Input: 91-91, Out: , Pred: 17
Input: 19-97, Out: -78, Pred: -66
Input: 68-92, Out: -24, Pred: -10
Input: 58-63, Out: -5, Pred: -2
Input: 55+40, Out: 95, Pred: 94

Full sequence accuracy: 10.000 %

--------------------------------------------------
Iteration 34
Epoch 1: Loss = 0.8388, Val Accuracy = 0.6807
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Input: 76+11, Out: 87, Pred: 7
Input: 67-2, Out: 65, Pred: 76
Input: 34-10, Out: 24, 

...

Epoch 1: Loss = 0.1054, Val Accuracy = 0.9860
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Input: 44-52, Out: -8, Pred: -8
Input: 70-23, Out: 47, Pred: 47
Input: 90+23, Out: 113, Pred: 113
Input: 7-99, Out: -92, Pred: -92
Input: 31-88, Out: -57, Pred: -57
Input: 32-82, Out: -50, Pred: -50
Input: 81+66, Out: 147, Pred: 147
Input: 69-4, Out: 65, Pred: 65
Input: 61-85, Out: -24, Pred: -24
Input: 16-57, Out: -41, Pred: -41
Input: 14+27, Out: 41, Pred: 41
Input: 88+59, Out: 147, Pred: 147
Input: 72+10, Out: 82, Pred: 82
Input: 39+72, Out: 111, Pred: 111
Input: 86-55, Out: 31, Pred: 31
Input: 47-95, Out: -48, Pred: -48
Input: 36-73, Out: -37, Pred: -37
Input: 73-, Out: 73, Pred: 73
Input: 83-16, Out: 67, Pred: 67
Input: 12+29, Out: 41, Pred: 41

Full sequence accuracy: 100.000 %

--------------------------------------------------
Iteration 162
Epoch 1: Loss = 0.1023, Val Accuracy = 0.9878
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Input: 12+68, Out: 80, Pred: 80
Input: 67+97, Out: 164, Pred: 164
Input: 

Epoch 1: Loss = 0.0896, Val Accuracy = 0.9894
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Input: 22+45, Out: 67, Pred: 67
Input: 52+80, Out: 132, Pred: 132
Input: 72+41, Out: 113, Pred: 113
Input: 64-34, Out: 30, Pred: 30
Input: 54+82, Out: 136, Pred: 136
Input: 90-18, Out: 72, Pred: 72
Input: 21-18, Out: 3, Pred: 7
Input: 77-27, Out: 50, Pred: 50
Input: 77-35, Out: 42, Pred: 42
Input: 42-29, Out: 13, Pred: 13
Input: 69-4, Out: 65, Pred: 65
Input: 26-53, Out: -27, Pred: -27
Input: 9+71, Out: 80, Pred: 80
Input: 69+33, Out: 102, Pred: 102
Input: 39+93, Out: 132, Pred: 132
Input: 80+34, Out: 114, Pred: 114
Input: 1-14, Out: -13, Pred: -13
Input: 28-11, Out: 17, Pred: 17
Input: 58+54, Out: 112, Pred: 112
Input: 85+67, Out: 152, Pred: 152

Full sequence accuracy: 95.000 %

--------------------------------------------------
Iteration 170
Epoch 1: Loss = 0.0883, Val Accuracy = 0.9887
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Input: 21-78, Out: -57, Pred: -57
Input: 56+31, Out: 87, Pred: 87
Input: 26+4

Epoch 1: Loss = 0.0788, Val Accuracy = 0.9910
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Input: 40+67, Out: 107, Pred: 107
Input: 13+58, Out: 71, Pred: 71
Input: 22-28, Out: -6, Pred: -6
Input: 20-38, Out: -18, Pred: -18
Input: 7+48, Out: 55, Pred: 55
Input: 8-17, Out: -9, Pred: -9
Input: 38-36, Out: 2, Pred: 2
Input: 32+82, Out: 114, Pred: 114
Input: 71-81, Out: -10, Pred: -10
Input: 35-84, Out: -49, Pred: -49
Input: 79-62, Out: 17, Pred: 17
Input: 27+26, Out: 53, Pred: 53
Input: 81-12, Out: 69, Pred: 69
Input: 55-24, Out: 31, Pred: 31
Input: -69, Out: -69, Pred: -69
Input: 29-93, Out: -64, Pred: -64
Input: 29-32, Out: -3, Pred: -3
Input: 99-77, Out: 22, Pred: 22
Input: 5+30, Out: 35, Pred: 35
Input: 18+89, Out: 107, Pred: 107

Full sequence accuracy: 100.000 %

--------------------------------------------------
Iteration 178
Epoch 1: Loss = 0.0762, Val Accuracy = 0.9916
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Input: 81-51, Out: 30, Pred: 30
Input: 48-80, Out: -32, Pred: -32
Input: 99-68, Ou

Epoch 1: Loss = 0.0678, Val Accuracy = 0.9933
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Input: 37-21, Out: 16, Pred: 16
Input: 10-92, Out: -82, Pred: -81
Input: 94+94, Out: 188, Pred: 188
Input: 4-41, Out: -37, Pred: -37
Input: 99-35, Out: 64, Pred: 64
Input: 6-11, Out: -5, Pred: -5
Input: 16-55, Out: -39, Pred: -39
Input: 29+47, Out: 76, Pred: 76
Input: 9-81, Out: -72, Pred: -72
Input: 36-85, Out: -49, Pred: -49
Input: 90+74, Out: 164, Pred: 164
Input: 45-85, Out: -40, Pred: -40
Input: 88-65, Out: 23, Pred: 23
Input: 48-44, Out: 4, Pred: 4
Input: 20+85, Out: 105, Pred: 105
Input: 83-6, Out: 77, Pred: 77
Input: 44-31, Out: 13, Pred: 13
Input: 76+3, Out: 79, Pred: 79
Input: 14-69, Out: -55, Pred: -55
Input: 46-41, Out: 5, Pred: 5

Full sequence accuracy: 95.000 %

--------------------------------------------------
Iteration 186
Epoch 1: Loss = 0.0662, Val Accuracy = 0.9928
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Input: 12-52, Out: -40, Pred: -40
Input: 7-17, Out: -10, Pred: -10
Input: 70-12, 

Epoch 1: Loss = 0.0594, Val Accuracy = 0.9943
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Input: 10+63, Out: 73, Pred: 73
Input: 69+36, Out: 105, Pred: 105
Input: 36-35, Out: 1, Pred: 1
Input: 13+16, Out: 29, Pred: 29
Input: 54-82, Out: -28, Pred: -28
Input: 25+48, Out: 73, Pred: 73
Input: 72-19, Out: 53, Pred: 53
Input: 33-73, Out: -40, Pred: -40
Input: 35-58, Out: -23, Pred: -23
Input: 42+58, Out: 100, Pred: 100
Input: 13-79, Out: -66, Pred: -66
Input: 4+30, Out: 34, Pred: 34
Input: 25+62, Out: 87, Pred: 87
Input: 30-39, Out: -9, Pred: -9
Input: 7-59, Out: -52, Pred: -52
Input: 29+46, Out: 75, Pred: 75
Input: 48-20, Out: 28, Pred: 28
Input: 8+22, Out: 30, Pred: 30
Input: 47+54, Out: 101, Pred: 101
Input: 81+46, Out: 127, Pred: 127

Full sequence accuracy: 100.000 %

--------------------------------------------------
Iteration 194
Epoch 1: Loss = 0.0572, Val Accuracy = 0.9947
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Input: 25-98, Out: -73, Pred: -73
Input: 81-57, Out: 24, Pred: 24
Input: 19-87

In [ ]:
import session_info
session_info.show(html=False)